In [16]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jun 17 18:32:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0              29W /  70W |    545MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [17]:
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

In [19]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["validation"] = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation", use_auth_token=True)

print(common_voice)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2554: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for patrickvonplaten/librispeech_asr_dummy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/patrickvonplaten/librispeech_asr_dummy.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    validation: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 73
    })
})


In [ ]:
common_voice = common_voice.remove_columns(["file", "speaker_id", "chapter_id", "id"])

print(common_voice)

DatasetDict({
    validation: Dataset({
        features: ['audio', 'text'],
        num_rows: 73
    })
})


In [ ]:
dataset = common_voice

In [ ]:
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 7.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.2 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801358 sha256=c123f6e5761c44c9eb0295203f9819cc7da72e05d6fc22c5aba6bd80d159dbfe
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper


In [ ]:
import whisper
from datasets import DatasetDict, load_dataset
from jiwer import cer
import numpy as np

# Initialize the Whisper model
model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 40.9MiB/s]


In [24]:
def transcribe_and_calculate_cer(model, dataset):
    transcriptions = []
    cers = []

    for example in dataset['validation']:
        # Get the audio file and the ground truth transcription
        audio = example['audio']['array']
        ground_truth = example['text']

        # Ensure audio data is in float32 format
        audio = np.array(audio, dtype=np.float32)  # Convert to float32

        # Transcribe the audio file
        print(f"Transcribing audio...")
        result = model.transcribe(audio)
        transcription = result["text"]

        # Calculate CER
        error_rate = cer(ground_truth.lower(), transcription.lower())

        # Append results
        transcriptions.append(transcription)
        cers.append(error_rate)

    return dataset, transcriptions, cers

In [26]:
result_dataset, transcriptions, cers = transcribe_and_calculate_cer(model, dataset)

Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribing audio...
Transcribi

In [27]:
result_dataset['validation'][0]

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/dfbece23564f422bc5794f3090902cd16d52d86767b746125ebc2ff3ea5f89ef/dev_clean/1272/135031/1272-135031-0000.flac',
  'array': array([-0.00018311, -0.00033569, -0.00021362, ..., -0.00323486,
         -0.00402832, -0.00393677]),
  'sampling_rate': 16000},
 'text': 'BECAUSE YOU WERE SLEEPING INSTEAD OF CONQUERING THE LOVELY ROSE PRINCESS HAS BECOME A FIDDLE WITHOUT A BOW WHILE POOR SHAGGY SITS THERE A COOING DOVE'}

In [28]:
cer = 0
for i in range(73):
    print(f"Audio {i+1}:")
    print(f"Ground Truth: {result_dataset['validation'][i]['text']}")
    print(f"Transcription: {transcriptions[i]}")
    print(f"CER: {cers[i]}\n")
    cer += cers[i]
print(f"Average CER: {cer/73}")

Audio 1:
Ground Truth: BECAUSE YOU WERE SLEEPING INSTEAD OF CONQUERING THE LOVELY ROSE PRINCESS HAS BECOME A FIDDLE WITHOUT A BOW WHILE POOR SHAGGY SITS THERE A COOING DOVE
Transcription:  Because you were sleeping instead of conquering, the lovely rose princess has become a fiddle without a bow, all porous shaggy sits there accuing dove.
CER: 0.087248322147651

Audio 2:
Ground Truth: HE HAS GONE AND GONE FOR GOOD ANSWERED POLYCHROME WHO HAD MANAGED TO SQUEEZE INTO THE ROOM BESIDE THE DRAGON AND HAD WITNESSED THE OCCURRENCES WITH MUCH INTEREST
Transcription:  He has gone and gone for good, answered polychrome who had managed to squeeze into the room beside the dragon and had witnessed the occurrences with much interest.
CER: 0.012422360248447204

Audio 3:
Ground Truth: I HAVE REMAINED A PRISONER ONLY BECAUSE I WISHED TO BE ONE AND WITH THIS HE STEPPED FORWARD AND BURST THE STOUT CHAINS AS EASILY AS IF THEY HAD BEEN THREADS
Transcription:  I have remained a prisoner only because I wish 

In [32]:

min_cer = min(cers)
print(f"BEST CER: {min_cer}")
print(f"Average CER: {cer/73}")


BEST CER: 0.012422360248447204
Average CER: 0.09490202272662603
